In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:
url = 'https://www.fantasypros.com/nfl/rankings/dynasty-overall.php'

In [23]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [24]:
response.status_code

200

In [25]:
table = soup.find('table', {'id': 'data'})

In [26]:
headers = []
for th in table.find_all('th'):
    headers.append(th.text.strip())

# Extract rows
rows = []
for tr in table.find_all('tr'):
    cells = tr.find_all(['td', 'th'])
    row = [cell.text.strip() for cell in cells]
    if row:  # avoid adding empty rows
        rows.append(row)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [15]:
df = pd.DataFrame(rows, columns=headers)

In [16]:
df

,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT,YDS,TD,FL,G,FPTS,FPTS/G,ROST
0,,,PASSING,RUSHING,MISC,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Rank,Player,CMP,ATT,PCT,YDS,Y/A,TD,INT,SACKS,ATT,YDS,TD,FL,G,FPTS,FPTS/G,ROST
2,1,Tua Tagovailoa (MIA),28,45,62.2,466,10.4,3,1,0,5,5,0,1,1,28.1,28.1,81.2%
3,2,Mac Jones (JAC),35,54,64.8,316,5.9,3,1,2,2,15,0,0,1,25.1,25.1,2.3%
4,3,Jordan Love (GB),15,27,55.6,245,9.1,3,0,1,3,12,0,0,1,23.0,23.0,85.7%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,138,Carson Wentz (KC),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.7%
140,139,Nate Sudfeld (DET),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0%
141,140,Michael Hiers (TB),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0%
142,141,Jake Browning (CIN),0,1,0.0,0,0.0,0,0,0,1,-1,0,0,1,-0.1,-0.1,20.8%
